# Utah HB 15 - Example Household Impacts

This notebook shows how Utah's proposed Medicaid expansion repeal (HB 15) would affect different household types.

**Key takeaway:** Whether someone falls into the "coverage gap" or transitions to ACA depends primarily on their income relative to the Federal Poverty Level (FPL):
- Below 100% FPL → Coverage gap (no ACA available)
- 100-138% FPL → Can transition to ACA subsidies

**Note:** These examples assume federal Medicaid work requirements (80+ hours/month) are in effect, as modeled for 2027. Parents with children under 13 are exempt from work requirements.

In [ ]:
from policyengine_us import Simulation
from policyengine_core.periods import instant
from policyengine_core.reforms import Reform

YEAR = 2027

def create_ut_medicaid_expansion_repeal():
    """Repeal Utah Medicaid expansion by setting income limit to -inf."""
    def modify_parameters(parameters):
        parameters.gov.hhs.medicaid.eligibility.categories.adult.income_limit.UT.update(
            start=instant(f"{YEAR}-01-01"),
            stop=instant("2100-12-31"),
            value=float("-inf"),
        )
        return parameters

    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)

    return reform

In [ ]:
def analyze_household(situation, name):
    """Analyze a household under baseline and reform."""
    baseline = Simulation(situation=situation)
    reformed = Simulation(situation=situation, reform=create_ut_medicaid_expansion_repeal())
    
    # Get results for each person
    people = list(situation["people"].keys())
    
    print(f"{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    for person in people:
        age = situation["people"][person].get("age", {}).get(YEAR, "N/A")
        
        # Baseline
        b_medicaid = baseline.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        b_ptc_elig = baseline.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        
        # Reform
        r_medicaid = reformed.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        r_ptc_elig = reformed.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        
        print(f"\n{person} (age {age}):")
        print(f"  Baseline:  Medicaid=${b_medicaid:,.0f}/yr, ACA eligible={b_ptc_elig}")
        print(f"  Reform:    Medicaid=${r_medicaid:,.0f}/yr, ACA eligible={r_ptc_elig}")
        
        if b_medicaid > 0 and r_medicaid == 0:
            if r_ptc_elig:
                print(f"  → LOSES MEDICAID, GAINS ACA ELIGIBILITY")
            else:
                print(f"  → FALLS INTO COVERAGE GAP (no coverage option)")
    
    # Tax unit level PTC
    b_ptc = baseline.calculate("premium_tax_credit", YEAR)[0]
    r_ptc = reformed.calculate("premium_tax_credit", YEAR)[0]
    
    print(f"\nHousehold Premium Tax Credit:")
    print(f"  Baseline: ${b_ptc:,.0f}/yr")
    print(f"  Reform:   ${r_ptc:,.0f}/yr")
    print()

## Household 1: Single Adult Below Poverty Line → Coverage Gap

A single adult earning $12,000/year (about 75% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they fall into the coverage gap because ACA subsidies only start at 100% FPL.

In [ ]:
household_1 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 12_000},
            "monthly_hours_worked": {YEAR: 100},  # Meets work requirement
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult"],
        }
    },
    "households": {
        "household": {
            "members": ["adult"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["adult"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult"],
        }
    },
}

analyze_household(household_1, "HOUSEHOLD 1: Single adult, $12k/yr (75% FPL) → COVERAGE GAP")

## Household 2: Single Adult Above Poverty Line → ACA Transition

A single adult earning $18,000/year (about 112% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they can transition to ACA marketplace coverage with premium subsidies.

In [ ]:
household_2 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 18_000},
            "monthly_hours_worked": {YEAR: 100},  # Meets work requirement
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult"],
        }
    },
    "households": {
        "household": {
            "members": ["adult"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["adult"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult"],
        }
    },
}

analyze_household(household_2, "HOUSEHOLD 2: Single adult, $18k/yr (112% FPL) → ACA TRANSITION")

## Household 3: Parent with Child Below Poverty Line → Coverage Gap for Parent

A single parent with one child, earning $15,000/year (about 68% FPL for family of 2). The parent loses Medicaid expansion and falls into the coverage gap. The child remains eligible for Medicaid/CHIP (children's eligibility is separate from expansion).

In [ ]:
household_3 = {
    "people": {
        "parent": {
            "age": {YEAR: 30},
            "employment_income": {YEAR: 15_000},
            "monthly_hours_worked": {YEAR: 100},  # Parent exempt (child under 13), but working anyway
        },
        "child": {
            "age": {YEAR: 8},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent", "child"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["parent", "child"],
        }
    },
    "households": {
        "household": {
            "members": ["parent", "child"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["parent", "child"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["parent"],
        }
    },
}

analyze_household(household_3, "HOUSEHOLD 3: Single parent + child, $15k/yr (68% FPL) → PARENT IN COVERAGE GAP")

## Household 4: Couple with Children Above Poverty Line → ACA Transition

A married couple with two children, earning $38,000/year (about 115% FPL for family of 4). Both parents lose Medicaid expansion but can transition to ACA. Children remain on Medicaid/CHIP.

In [ ]:
household_4 = {
    "people": {
        "parent1": {
            "age": {YEAR: 40},
            "employment_income": {YEAR: 38_000},
            "monthly_hours_worked": {YEAR: 160},  # Full-time worker
        },
        "parent2": {
            "age": {YEAR: 38},
            "monthly_hours_worked": {YEAR: 0},  # Exempt (children under 13)
        },
        "child1": {
            "age": {YEAR: 12},
        },
        "child2": {
            "age": {YEAR: 7},
        }
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "households": {
        "household": {
            "members": ["parent1", "parent2", "child1", "child2"],
            "state_code": {YEAR: "UT"},
        }
    },
    "families": {
        "family": {
            "members": ["parent1", "parent2", "child1", "child2"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["parent1", "parent2"],
        }
    },
}

analyze_household(household_4, "HOUSEHOLD 4: Married couple + 2 kids, $38k/yr (115% FPL) → ACA TRANSITION")

## Visualizing the Coverage Gap

The graphs below show how health coverage changes across the income spectrum under Utah HB 15.

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# =============================================================================
# PolicyEngine v2 Chart Styling (matches policyengine-app-v2)
# =============================================================================
PE_COLORS = {
    'primary_500': '#2C9B99',   # Teal - reform/positive (primary[500])
    'primary_600': '#268786',   # Teal darker
    'primary_alpha': 'rgba(44, 155, 153, 0.3)',  # primary with alpha for fills
    'gray_600': '#4B5563',      # Gray - baseline/negative
    'gray_400': '#9CA3AF',      # Lighter gray for reference lines
    'gray_200': '#E5E7EB',      # Very light gray for grid
    'text': '#1F2937',          # Dark text
    'background': '#FFFFFF',    # White
}

# Chart font (Inter is PE default, fallback to system sans-serif)
PE_FONT = dict(family="Inter, system-ui, sans-serif", size=12, color=PE_COLORS['text'])

# Generate data across income spectrum for a single adult
incomes = np.arange(5000, 30001, 1000)
fpl_2027 = 16334  # FPL for single person in 2027

baseline_medicaid = []
baseline_ptc = []
reform_medicaid = []
reform_ptc = []

print("Calculating benefits across income levels...")
for income in incomes:
    situation = {
        "people": {"adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: int(income)},
            "monthly_hours_worked": {YEAR: 100},
        }},
        "tax_units": {"tax_unit": {"members": ["adult"]}},
        "spm_units": {"spm_unit": {"members": ["adult"]}},
        "households": {"household": {"members": ["adult"], "state_code": {YEAR: "UT"}}},
        "families": {"family": {"members": ["adult"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
    }
    
    base = Simulation(situation=situation)
    ref = Simulation(situation=situation, reform=create_ut_medicaid_expansion_repeal())
    
    baseline_medicaid.append(base.calculate("medicaid", YEAR, map_to="person")[0])
    baseline_ptc.append(base.calculate("premium_tax_credit", YEAR)[0])
    reform_medicaid.append(ref.calculate("medicaid", YEAR, map_to="person")[0])
    reform_ptc.append(ref.calculate("premium_tax_credit", YEAR)[0])

print("Done!")

Calculating benefits across income levels...
Done!


In [24]:
# Convert to arrays and calculate FPL percentages
incomes_arr = np.array(incomes)
fpl_pct = (incomes_arr / fpl_2027) * 100
baseline_medicaid = np.array(baseline_medicaid)
baseline_ptc = np.array(baseline_ptc)
reform_medicaid = np.array(reform_medicaid)
reform_ptc = np.array(reform_ptc)

# Calculate total health benefits
baseline_total = baseline_medicaid + baseline_ptc
reform_total = reform_medicaid + reform_ptc

In [ ]:
# Graph 1: Health Benefits by Income - Baseline vs Reform (side by side)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Current Law (with Medicaid Expansion)", "After HB 15 (Expansion Repealed)"),
    horizontal_spacing=0.1,
)

# Left: Baseline
fig.add_trace(go.Scatter(
    x=fpl_pct, y=baseline_total/1000,
    mode='lines',
    fill='tozeroy',
    fillcolor=PE_COLORS['primary_alpha'],
    line=dict(color=PE_COLORS['primary_500'], width=2),
    name='Total Benefits',
    hovertemplate='<b>%{x:.0f}% FPL</b><br>Benefits: $%{y:.1f}k<extra></extra>',
), row=1, col=1)

# Right: Reform
fig.add_trace(go.Scatter(
    x=fpl_pct, y=reform_total/1000,
    mode='lines',
    fill='tozeroy',
    fillcolor=PE_COLORS['primary_alpha'],
    line=dict(color=PE_COLORS['primary_500'], width=2),
    name='Total Benefits',
    showlegend=False,
    hovertemplate='<b>%{x:.0f}% FPL</b><br>Benefits: $%{y:.1f}k<extra></extra>',
), row=1, col=2)

# Add reference lines
for col in [1, 2]:
    fig.add_vline(x=100, line=dict(color=PE_COLORS['gray_400'], width=1, dash='dash'), row=1, col=col)
    fig.add_vline(x=138, line=dict(color=PE_COLORS['gray_400'], width=1, dash='dot'), row=1, col=col)

# Layout
fig.update_layout(
    font=PE_FONT,
    height=400,
    showlegend=True,
    legend=dict(x=0.02, y=0.98, xanchor='left', yanchor='top'),
    margin=dict(t=40, b=60, l=60, r=20),
    plot_bgcolor=PE_COLORS['background'],
    paper_bgcolor=PE_COLORS['background'],
)

fig.update_xaxes(title_text="Income (% of Federal Poverty Level)", range=[30, 185], fixedrange=True)
fig.update_yaxes(title_text="Annual Benefits ($1,000s)", range=[0, 15], fixedrange=True)

fig.show()
fig.write_image("hb15_benefits_comparison.png", scale=2)

In [ ]:
# Graph 2: Change in Benefits (Bar chart - matches PE distributional impact style)
benefit_change = reform_total - baseline_total

# Color bars based on whether positive or negative (gray for negative, teal for positive)
bar_colors = [PE_COLORS['gray_600'] if benefit_change[i] < 0 else PE_COLORS['primary_500'] 
              for i in range(len(incomes))]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=fpl_pct,
    y=benefit_change/1000,
    marker_color=bar_colors,
    hovertemplate='<b>%{x:.0f}% FPL</b><br>Change: $%{y:.1f}k<extra></extra>',
))

# Reference lines
fig.add_vline(x=100, line=dict(color=PE_COLORS['gray_400'], width=1, dash='dash'),
              annotation_text="100% FPL", annotation_position="top")
fig.add_vline(x=138, line=dict(color=PE_COLORS['gray_400'], width=1, dash='dot'),
              annotation_text="138% FPL", annotation_position="top")
fig.add_hline(y=0, line=dict(color=PE_COLORS['text'], width=0.5))

fig.update_layout(
    title=dict(
        text="Impact of Utah HB 15: Change in Health Benefits by Income",
        font=dict(size=14, color=PE_COLORS['text']),
    ),
    font=PE_FONT,
    height=450,
    showlegend=False,
    xaxis=dict(
        title="Income (% of Federal Poverty Level)",
        range=[30, 185],
        fixedrange=True,
    ),
    yaxis=dict(
        title="Change in Annual Benefits ($1,000s)",
        fixedrange=True,
    ),
    margin=dict(t=60, b=60, l=60, r=20),
    plot_bgcolor=PE_COLORS['background'],
    paper_bgcolor=PE_COLORS['background'],
    bargap=0.1,
)

fig.show()
fig.write_image("hb15_benefit_change.png", scale=2)

## Key Takeaways from the Graphs

1. **The Coverage Gap** (red zone in Graph 2): People below 100% FPL lose ~$8,000/year in Medicaid benefits with NO replacement coverage available. This is the most severe impact of repealing expansion.

2. **ACA Transition Zone** (100-138% FPL): People in this range can transition to ACA subsidies, which partially offset the loss of Medicaid. The Premium Tax Credit provides ~$8,000-$12,000/year in this income range.

3. **Above 138% FPL**: No change - these individuals were never eligible for expansion Medicaid.

**Bottom line:** The lower your income, the worse the impact of HB 15. Those with the least resources face the greatest loss.